In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
abbr = 'SC'

In [4]:
file = 'south_carolina2018.xlsx'

Read in federal level data

In [5]:
fiscal = pd.read_sas('../../data/fiscal2018', format = 'sas7bdat', encoding='iso-8859-1')

Generate list of districts in the state in the federal data

In [6]:
state_fiscal = fiscal[(fiscal['STABBR'] == abbr) & (fiscal['GSHI'] == '12')]

In [7]:
len(state_fiscal)

97

Read in state level data

In [8]:
state_grads = pd.read_excel('../../data/state_data_raw/' + file)

In [9]:
state_grads.head()

,DISTRICT,SCHOOL,BEDSCODE,GNUMERATOR_OVERALL,GDENOM_OVERALL,GPERCENT_OVERALL,GNUMERATOR_GENDER_M,GDENOM_GENDER_M,GPERCENT_GENDER_M,GNUMERATOR_GENDER_F,...,GPERCENT_HOMELESS_YES,GNUMERATOR_HOMELESS_NO,GDENOM_HOMELESS_NO,GPERCENT_HOMELESS_NO,GNUMERATOR_FOSTER_YES,GDENOM_FOSTER_YES,GPERCENT_FOSTER_YES,GNUMERATOR_FOSTER_NO,GDENOM_FOSTER_NO,GPERCENT_FOSTER_NO
0,Abbeville,ABBEVILLE HIGH,160001,117.0,134.0,87.313433,70.0,81.0,86.419753,47.0,...,-1.0,115.0,131.0,87.786260,-1.0,-1.0,-1.0,115.0,132.0,87.121212
1,Abbeville,DIXIE HIGH,160003,65.0,83.0,78.313253,26.0,35.0,74.285714,39.0,...,-1.0,64.0,79.0,81.012658,NaN,NaN,NaN,65.0,83.0,78.313253
2,Abbeville,DISTRICT NO CHARTER,160888,182.0,217.0,83.870968,96.0,116.0,82.758621,86.0,...,-1.0,179.0,210.0,85.238095,-1.0,-1.0,-1.0,180.0,215.0,83.720930
3,Abbeville,DISTRICT WITH CHARTER,160999,182.0,217.0,83.870968,96.0,116.0,82.758621,86.0,...,-1.0,179.0,210.0,85.238095,-1.0,-1.0,-1.0,180.0,215.0,83.720930
4,Aiken,SILVER BLUFF HIGH,201001,133.0,152.0,87.500000,73.0,86.0,84.883721,60.0,...,-1.0,129.0,145.0,88.965517,NaN,NaN,NaN,133.0,152.0,87.500000


Filter results.

In [10]:
state_grads = state_grads[(state_grads['SCHOOL'] == 'DISTRICT WITH CHARTER')]

Select and rename columns.

In [11]:
state_grads = state_grads[['DISTRICT', 
                           'GDENOM_OVERALL', 
                           'GPERCENT_OVERALL']]

In [12]:
state_grads.columns = ['District Name', 'Total', 'Graduation Rate']

In [13]:
state_grads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 3 to 418
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   District Name    88 non-null     object 
 1   Total            88 non-null     float64
 2   Graduation Rate  88 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.8+ KB


Convert data types.

In [14]:
# state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('\*', '')

In [14]:
state_grads['District Name'] = state_grads['District Name'].astype(str)
state_grads['Total'] = pd.to_numeric(state_grads['Total'])
state_grads['Graduation Rate'] = pd.to_numeric(state_grads['Graduation Rate']) / 100

Check for matches and non-matches in the two lists. 

In [16]:
# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.upper().str.strip()
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.upper().str.strip()

In [37]:
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' 0', ' ')
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(r' 1$', '')

state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(r' 1$', '')

In [38]:
matches = [name for name in list(state_grads['District Name']) if name in list(state_fiscal['NAME'])]
matches.sort()
len(matches)

75

In [39]:
A = [name for name in list(state_grads['District Name']) if name not in list(state_fiscal['NAME'])]
A.sort()
A

['Abbeville',
 "Governor's Schools",
 "Governor's Schools",
 'Lexington/Richland  5',
 'SC Department of Corrections',
 'SC Department of Juvenile Justice',
 'SC Public Charter District',
 'SC School for the Deaf and the Blind',
 'STATEWIDE',
 'York 1 (York)',
 'York 2 (Clover)',
 'York 3 (Rock Hill)',
 'York 4 (Fort Mill)']

In [40]:
B = [name for name in list(state_fiscal['NAME']) if name not in list(state_grads['District Name'])]
B.sort()
B

['Abbeville 60',
 'Anderson 80',
 'Barnwell 80',
 'Beaufort 80',
 'Clarendon 80',
 'Deaf & Blind School',
 'Dept of Juvenile Justice',
 'Dillon 80',
 'Dorchester 80',
 "Governor' School for Science and Mathematics",
 "Governor's School for the Arts and Humanities",
 'Greenwood 80',
 'Lexington 5',
 'Orangeburg 80',
 'SC Public Charter School District',
 'Spartanburg 80',
 'Spartanburg 81',
 'Spartanburg 82',
 'York',
 'York 2',
 'York 3',
 'York 4']

Make any additional matches I can find.

In [43]:
state_fiscal_rename = {
    'Abbeville 60' : 'Abbeville',
    'Deaf & Blind School' : 'SC School for the Deaf and the Blind',
    'Lexington 5' : 'Lexington/Richland  5',
    'SC Public Charter School District' : 'SC Public Charter District',
    'York' : 'York 1 (York)',
    'York 2' : 'York 2 (Clover)',
    'York 3' : 'York 3 (Rock Hill)',
    'York 4' : 'York 4 (Fort Mill)'
}

In [44]:
state_fiscal = state_fiscal.replace(state_fiscal_rename)

Merge federal and state data, keeping only matches between the two. 

In [23]:
state_grads_merged = pd.merge(state_fiscal, state_grads, how='inner', left_on='NAME', right_on='District Name')

Save cleaned data. 

In [24]:
state_grads_merged.to_csv('../../data/state_data_merged/' + abbr + '.csv', index=False)